In [2]:
conda activate lenci_enviroment



Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import surprise
from surprise import SVD, Reader
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (
    rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, recall_at_k
)
from collections import defaultdict

In [4]:
# =================== 1. Preparar dataset ===================
file_path = "C:\\Users\\vlenc\\OneDrive\\Documentos\\ml-latest-small\\ratings.csv"
movies_file = "C:\\Users\\vlenc\\OneDrive\\Documentos\\ml-latest-small\\movies.csv"

df = pd.read_csv(file_path)
df = df[["userId", "movieId", "rating"]]
# Padronizar nomes de colunas para Recommenders
df = df.rename(columns={"userId": "userID", "movieId": "itemID", "rating": "rating"})

movies = pd.read_csv(movies_file)
movies = movies.rename(columns={"movieId": "itemID", "title": "title"})
# movies = movies.merge(df.groupby("itemID").size().reset_index(name="num_ratings"), on="itemID")
# movies = movies.sort_values("num_ratings", ascending=False).reset_index(drop=True)

In [ ]:
# =================== 2. Criar treino/teste ===================

# A seed é o que faz a base de treino/teste sempre ser a mesma.
train_df, test_df = python_random_split(df, 0.8, seed=412)

# Criar dataset do Surprise
reader = Reader(rating_scale=(1, 5))

# Adaptar DataFrame 'train_df' para ser utilizado na função 'SVD'
trainset = surprise.Dataset.load_from_df(train_df, reader).build_full_trainset()

In [22]:
test_df.head(5)

,userID,itemID,rating
25145,177,2078,3.0
42535,288,2286,2.0
15959,104,436,1.5
70015,448,92198,2.0
97464,606,593,4.5


In [23]:
# =================== 3. Treinar SVD ===================
svd = SVD(n_factors=200, n_epochs=30, random_state=42, verbose=True)
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29


In [24]:
# =================== 4. Criar predições  ===================
predictions = predict(svd, test_df, usercol="userID", itemcol="itemID")
predictions.head(5)

,userID,itemID,prediction
0,177,2078,3.962883
1,288,2286,3.082872
2,104,436,3.431700
3,448,92198,2.890218
4,606,593,4.510129


In [25]:
# É utilizado para avaliar as métricas de ranking (MAP, NDCG, Precision@K, Recall@K).

all_predictions = compute_ranking_predictions(
    svd, train_df, usercol="userID", itemcol="itemID", remove_seen=True
)
all_predictions.head()

# No bloco de código seguinte é feito uma comparação entre as predições e as notas reais

,userID,itemID,prediction
1,1,2,4.187954
3,1,4,3.687404
4,1,5,3.660560
6,1,7,3.571751
7,1,8,3.797254


In [26]:
# Verificar se a base de dados tá realmente randomizando a cada vez que roda, já que o resultado é sempre o mesmo.

# “O modelo acerta a nota que o usuário deu para o filme?”
eval_rmse = rmse(test_df, predictions)
eval_mae = mae(test_df, predictions)
eval_rsquared = rsquared(test_df, predictions)
eval_exp_var = exp_var(test_df, predictions)

# O modelo está colocando os filmes certos (que o usuário realmente gosta) nas primeiras posições das recomendações?
eval_map = map_at_k(test_df, all_predictions, col_prediction="prediction", k=10)
eval_ndcg = ndcg_at_k(test_df, all_predictions, col_prediction="prediction", k=10)
eval_precision = precision_at_k(
    test_df, all_predictions, col_prediction="prediction", k=10
)
eval_recall = recall_at_k(test_df, all_predictions, col_prediction="prediction", k=10)


print(
    "RMSE:\t\t%f" % eval_rmse,
    "MAE:\t\t%f" % eval_mae,
    "rsquared:\t%f" % eval_rsquared,
    "exp var:\t%f" % eval_exp_var,
    sep="\n",
)

print("----")

print(
    "MAP:\t\t%f" % eval_map,
    "NDCG:\t\t%f" % eval_ndcg,
    "Precision@K:\t%f" % eval_precision,
    "Recall@K:\t%f" % eval_recall,
    sep="\n",
)

RMSE:		0.879654
MAE:		0.676283
rsquared:	0.294669
exp var:	0.294731
----
MAP:		0.026977
NDCG:		0.061042
Precision@K:	0.053934
Recall@K:	0.019723


In [27]:
# =================== 5. Recomendação para usuário final ===================
# Exemplo: usuário final com userID = 4
user_id = 4

# Obter todas as avaliações desse usuário no dataset
user_ratings = df[df["userID"] == user_id]

# Conjunto de filmes que o usuário já avaliou
user_movies = set(user_ratings["itemID"])

# Todos os filmes do dataset
all_movies = set(df["itemID"])

# Filmes que o usuário ainda não avaliou
movies_to_predict = list(all_movies - user_movies)

# Criar anti-testset para esse usuário
anti_testset_user = [(user_id, iid, 0) for iid in movies_to_predict]  # 0 é placeholder

print(anti_testset_user)

# Fazer predição das notas
predictions_user = svd.test(anti_testset_user)

print(predictions_user)

# Função para pegar Top-N recomendações
def get_top_n(predictions, n=10):
    from collections import defaultdict
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid in top_n:
        top_n[uid].sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = top_n[uid][:n]
    return top_n

top_recommendations = get_top_n(predictions_user, n=10)

# Exibir recomendações
print("\n🎬 Top 10 filmes recomendados para o usuário %d:" % user_id)
for item_id, pred_rating in top_recommendations[user_id]:
    title = movies[movies["itemID"] == item_id]["title"].values[0]
    print(f"{title} (nota estimada: {pred_rating:.2f})")

[(4, 1, 0), (4, 2, 0), (4, 3, 0), (4, 4, 0), (4, 5, 0), (4, 6, 0), (4, 7, 0), (4, 8, 0), (4, 9, 0), (4, 10, 0), (4, 11, 0), (4, 12, 0), (4, 13, 0), (4, 14, 0), (4, 15, 0), (4, 16, 0), (4, 17, 0), (4, 18, 0), (4, 19, 0), (4, 20, 0), (4, 22, 0), (4, 23, 0), (4, 24, 0), (4, 25, 0), (4, 26, 0), (4, 27, 0), (4, 28, 0), (4, 29, 0), (4, 131098, 0), (4, 31, 0), (4, 131104, 0), (4, 34, 0), (4, 30, 0), (4, 36, 0), (4, 32799, 0), (4, 38, 0), (4, 39, 0), (4, 40, 0), (4, 41, 0), (4, 65577, 0), (4, 43, 0), (4, 44, 0), (4, 46, 0), (4, 48, 0), (4, 65585, 0), (4, 50, 0), (4, 49, 0), (4, 53, 0), (4, 54, 0), (4, 55, 0), (4, 65588, 0), (4, 57, 0), (4, 98361, 0), (4, 60, 0), (4, 61, 0), (4, 62, 0), (4, 65596, 0), (4, 64, 0), (4, 65, 0), (4, 66, 0), (4, 131130, 0), (4, 68, 0), (4, 69, 0), (4, 70, 0), (4, 71, 0), (4, 72, 0), (4, 73, 0), (4, 74, 0), (4, 75, 0), (4, 76, 0), (4, 77, 0), (4, 78, 0), (4, 79, 0), (4, 80, 0), (4, 81, 0), (4, 82, 0), (4, 83, 0), (4, 85, 0), (4, 86, 0), (4, 87, 0), (4, 88, 0), (4, 89

In [28]:
# Resultados de 'all_predictions' para o usuário 4
df_user1 = pd.DataFrame(all_predictions)
res = df_user1[df_user1["userID"] == 4].sort_values(by="prediction", ascending=False)

# Adicionar o título do filme
res = res.merge(movies[["itemID", "title"]], on="itemID", how="left")

# Mostrar os top 50
res.head(50)


,userID,itemID,prediction,title
0,4,953,4.996993,It's a Wonderful Life (1946)
1,4,912,4.914186,Casablanca (1942)
2,4,1148,4.720508,Wallace & Gromit: The Wrong Trousers (1993)
3,4,899,4.706069,Singin' in the Rain (1952)
4,4,3468,4.682674,"Hustler, The (1961)"
5,4,96488,4.633326,Searching for Sugar Man (2012)
6,4,142488,4.584117,Spotlight (2015)
7,4,1228,4.539851,Raging Bull (1980)
8,4,1262,4.525952,"Great Escape, The (1963)"
9,4,111,4.514422,Taxi Driver (1976)


In [ ]:
# =================== 6. Simulação de novo usuário ===================
from collections import defaultdict

# Definir o ID do novo usuário (não presente no dataset original)
novo_user_id = df["userID"].max() + 5  # por exemplo, 610

# Mostrar filmes populares para o usuário avaliar (pode ajustar critério de seleção)
# Selecionar filmes aleatórios ou os mais populares
avaliacoes_usuario = []
MAX_AVALIACOES = 20
contador = 0

print(f"\n📝 Novo usuário ({novo_user_id}) vai avaliar {MAX_AVALIACOES} filmes.\n")

for _, row in movies.sample(frac=1).iterrows():  # mistura os filmes
    if contador >= MAX_AVALIACOES:
        break
    
    movie_id = row["itemID"]
    title = row["title"]
    # Pega apenas o primeiro gênero
    first_genre = row["genres"].split("|")[0] if "genres" in row else "Desconhecido"
    
    resposta = input(f"Você assistiu '{title}' (Gênero: {first_genre})? Nota 1-5 (0=Não assistiu): ")
    
    try:
        nota = int(resposta)
        if nota == 0:
            continue  # usuário não assistiu, ignora
        if nota < 1 or nota > 5:
            print("Nota inválida, use 1-5.")
            continue
    except:
        print("Resposta inválida, tente novamente.")
        continue
    
    avaliacoes_usuario.append((novo_user_id, movie_id, nota))
    contador += 1

print(f"\n✅ Coletadas {len(avaliacoes_usuario)} avaliações do novo usuário.\n")

# Criar DataFrame temporário com avaliações do novo usuário
df_novo_usuario = pd.DataFrame(avaliacoes_usuario, columns=["userID", "itemID", "rating"])

# Construir anti-testset apenas para os filmes que o usuário ainda não avaliou
user_movies = set(df_novo_usuario["itemID"])
all_movies = set(df["itemID"])
movies_to_predict = list(all_movies - user_movies)

anti_testset_user = [(novo_user_id, iid, 0) for iid in movies_to_predict]

# Predição SVD para o novo usuário
predicoes_novo_usuario = svd.test(anti_testset_user)

# Função para pegar top-N recomendações
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid in top_n:
        top_n[uid].sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = top_n[uid][:n]
    return top_n

top_recommendations = get_top_n(predicoes_novo_usuario, n=10)

# Mostrar resultados com título e primeiro gênero
print(f"\n🎬 Top 10 recomendações para o usuário {novo_user_id}:")
for item_id, pred_rating in top_recommendations[novo_user_id]:
    title = movies[movies["itemID"] == item_id]["title"].values[0]
    first_genre = movies[movies["itemID"] == item_id]["genres"].values[0].split("|")[0]
    print(f"{title} (Gênero: {first_genre}, nota estimada: {pred_rating:.2f})")



📝 Novo usuário (615) vai avaliar 20 filmes.



In [18]:
# =================== 7. Fold-in corrigido: só mostrar filmes do trainset e evitar fallback desnecessário ====
import numpy as np

MAX_AVALIACOES_NOVO = 20
MIN_AVALIACOES_PARA_FOLDIN = 5   # você pode ajustar
LAMBDA_REG = 0.1
TOP_K = 10

novo_user_id = int(df["userID"].max()) + 1
print(f"\n=== Fold-in: novo usuário {novo_user_id} ===")
print(f"Iremos coletar até {MAX_AVALIACOES_NOVO} avaliações (1-5). Responda 0 se não assistiu.)\n")

# --- 1) Encontrar os raw item IDs que aparecem no trainset (raw ids como strings) ---
train_raw_ids = [trainset.to_raw_iid(i) for i in range(trainset.n_items)]
# Filtrar o DataFrame de filmes para conter apenas itens presentes no trainset
movies_in_train = movies[movies["itemID"].astype(str).isin(train_raw_ids)].copy()

if movies_in_train.empty:
    raise RuntimeError("Nenhum filme do DataFrame de filmes foi encontrado no trainset. Verifique os IDs.")

# amostrar filmes a mostrar ao usuário (embaralha com random_state para reprodutibilidade)
movies_to_show = movies_in_train.sample(frac=1, random_state=42).reset_index(drop=True)

avaliacoes = []
contador = 0

for _, row in movies_to_show.iterrows():
    if contador >= MAX_AVALIACOES_NOVO:
        break
    raw_iid = str(row["itemID"])  # usar string para compatibilidade com trainset mappings
    title = row["title"]
    first_genre = row["genres"].split("|")[0] if "genres" in row and isinstance(row["genres"], str) else "Desconhecido"

    resposta = input(f"Nota para '{title}' (Gênero: {first_genre}) [1-5] (0 = não assisti): ")
    try:
        nota = int(resposta)
    except:
        print("Entrada inválida, tente novamente.")
        continue
    if nota == 0:
        continue
    if nota < 1 or nota > 5:
        print("Nota inválida, use 1-5.")
        continue

    # aqui garantimos que o raw_iid está no trainset, pois movies_to_show foi filtrado
    avaliacoes.append((raw_iid, float(nota)))
    contador += 1

print(f"\nColetadas {len(avaliacoes)} avaliações do novo usuário.\n")

# Mapear avaliações para inner ids (agora não deve lançar erro, pois filtramos)
inner_ids = []
ratings = []
for raw_iid, r in avaliacoes:
    try:
        inner = trainset.to_inner_iid(raw_iid)
    except ValueError:
        # safety: deveria não acontecer, porque filtramos, mas apenas no caso improvável, pular
        continue
    inner_ids.append(inner)
    ratings.append(r)

# Se ainda assim avaliações válidas forem poucas, avisar e usar fallback (apenas aviso)
if len(inner_ids) < MIN_AVALIACOES_PARA_FOLDIN:
    print("Avaliações válidas insuficientes para fold-in (usuário avaliou poucos filmes ou cancelou).")
    print("Considere pedir mais avaliações ou usar fallback por popularidade.\n")
    # aqui apenas mostramos um fallback por popularidade (pode manter / customizar)
    pop = df.groupby("itemID").size().reset_index(name="num_ratings")
    pop = pop.merge(movies[["itemID", "title", "genres"]], on="itemID", how="left")
    pop = pop.sort_values("num_ratings", ascending=False).head(TOP_K)
    print("Top por popularidade (fallback):")
    for _, row in pop.iterrows():
        first_genre = row["genres"].split("|")[0] if isinstance(row.get("genres", None), str) else "Desconhecido"
        print(f"{row['title']} (Gênero: {first_genre})")
else:
    # construir Q e y (resíduos)
    mu = trainset.global_mean
    k = svd.n_factors
    Q = np.array([svd.qi[i] for i in inner_ids])     # shape (m, k)
    bi = np.array([svd.bi[i] for i in inner_ids])    # item biases for rated items
    y = np.array(ratings) - mu - bi                  # residuals

    # resolver (Q^T Q + lambda I) p_u = Q^T y
    A = Q.T.dot(Q) + LAMBDA_REG * np.eye(k)
    b = Q.T.dot(y)
    try:
        p_u = np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        p_u = np.linalg.pinv(A).dot(b)

    # estimar bias do usuário
    residuals = y - Q.dot(p_u)
    b_u = float(np.mean(residuals)) if residuals.size > 0 else 0.0

    # predição para todos os items do trainset
    preds = []
    for inner_j in range(trainset.n_items):
        qj = svd.qi[inner_j]
        bj = svd.bi[inner_j]
        est = mu + bj + b_u + p_u.dot(qj)
        raw_j = trainset.to_raw_iid(inner_j)  # string
        preds.append((raw_j, est))

    # remover itens que o usuário já avaliou (raw ids strings)
    avaliados_raw = set([raw for raw, _ in avaliacoes])
    preds = [p for p in preds if p[0] not in avaliados_raw]

    # ordenar e pegar top K
    preds.sort(key=lambda x: x[1], reverse=True)
    topk = preds[:TOP_K]

    # exibir resultados com título e primeiro gênero
    print(f"\n🎬 Top-{TOP_K} recomendações para o novo usuário (fold-in):\n")
    for raw_j, est in topk:
        # raw_j é string, casar com movies.itemID convertendo também para string
        row = movies[movies["itemID"].astype(str) == raw_j]
        if not row.empty:
            title = row.iloc[0]["title"]
            genres = row.iloc[0]["genres"] if "genres" in row.columns else ""
            first_genre = genres.split("|")[0] if isinstance(genres, str) and genres else "Desconhecido"
            print(f"{title} (Gênero: {first_genre}) — nota estimada: {est:.2f}")
        else:
            print(f"{raw_j} — nota estimada: {est:.2f}")

print("\n(Fim do fold-in)\n")



=== Fold-in: novo usuário 611 ===
Iremos coletar até 20 avaliações (1-5). Responda 0 se não assistiu.)



Entrada inválida, tente novamente.
Nota inválida, use 1-5.

Coletadas 20 avaliações do novo usuário.


🎬 Top-10 recomendações para o novo usuário (fold-in):

Blade Runner (1982) (Gênero: Action) — nota estimada: 6.42
Cool Hand Luke (1967) (Gênero: Drama) — nota estimada: 5.89
Schindler's List (1993) (Gênero: Drama) — nota estimada: 5.85
Moulin Rouge (2001) (Gênero: Drama) — nota estimada: 5.75
Rise of the Footsoldier (2007) (Gênero: Action) — nota estimada: 5.75
Terminator, The (1984) (Gênero: Action) — nota estimada: 5.68
Topaz (1969) (Gênero: Thriller) — nota estimada: 5.65
Be Kind Rewind (2008) (Gênero: Comedy) — nota estimada: 5.61
Futurama: Bender's Big Score (2007) (Gênero: Animation) — nota estimada: 5.61
English Patient, The (1996) (Gênero: Drama) — nota estimada: 5.61

(Fim do fold-in)

